In [ ]:
from nb_shared import *

## Cluster for batch processing

In [ ]:
cluster = start_cluster(n_workers=20, local=False)

## DB to store SWANN zonal values

In [ ]:
zone_db = ZoneDB(DB_CONNECTION_INFO)

## Target Zones

In [ ]:
target_ch5ids = [
    'ANBC2',
    'RCYC2',
    'ALEC2',
    'ENMC2',
    'WBRW4',
    'BPPA3',
    'OAWU1'
]

In [ ]:
target_zones = zone_db.zone_in_ch5_ids(target_ch5ids)

## SWANN NetCDF files

In [ ]:
swann_xr = xr.open_mfdataset(SWANN_HOME_DIR + '*.nc', parallel=True)

## Extract SWE for each year from the NetCDF files

In [ ]:
swann_swe = {
    cz.zone: swann_swe_for_zone(swann_xr, cz.zone, zone_db) for cz in target_zones
}

In [ ]:
cluster.shutdown()

## Prepare dataframes and write to DB

In [ ]:
for czone in target_zones:
    # Prepare DB to match table columns
    swann_swe[czone.zone].rename(columns={'SWE': 'swe'}, inplace=True)
    swann_swe[czone.zone].index.names = ['z_date']
    swann_swe[czone.zone]['cbrfc_zone_id'] = czone.gid
    # Write
    zone_db.write(swann_swe[czone.zone], 'swann_zonal_swe')